In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# Author: RenzeLou
# use this script to preprocess all .tsv file to .json format and generate .source_vocab for each split(train/dev/test)
# we don't do lowering because we use bert-base-cased

import json,csv
import numpy as np
import pandas as pd
import re

In [2]:
train_file = []
dev_file = []
test_file = []
labels = []
desolate = 0

In [3]:
def break_up(error_str:str)->list:
    ans = []
    lines=error_str.strip().split("\n")
    for line in lines:
        res=line.split("\t")
        ans += res
    return ans

In [4]:
def fix_glue_error(data:list,stand:int):
    ans = []
    for element in data:
        res=break_up(element)
        ans += res
    assert len(ans) % stand == 0
    new_data = []
    for i in range(len(ans)//stand):
        new_element=ans[i*stand:(i+1)*stand]
        new_data.append(new_element)
    return new_data

In [5]:
desolate = 0
with open("./train.tsv","r",encoding="utf-8") as train:
    t=csv.reader(train,delimiter='\t')
    for i,line in enumerate(t):
        if i==0:
            print("skip first line")
            continue
#         try:
#             assert len(line) == 4
#         except:
# #             print("error_line:",line)
#             new_lines=fix_glue_error(line,4)
#             for i,new_line in enumerate(new_lines):
#                 print("line fixed:",new_line)
#                 lb=new_line[3]
#                 if lb not in labels:
#                     labels.append(lb)
#                 label = labels.index(lb)   
#                 instance = dict()
#                 instance["index"],instance['seq1'],instance['seq2'],instance["label"]=new_line[0],new_line[1],new_line[2],label
#                 train_file.append(instance)
#             continue
#         lb=line[3]
#         if lb not in labels:
#             labels.append(lb)
#         label = labels.index(lb)
#         instance = dict()
#         instance["index"],instance['seq1'],instance['seq2'],instance["label"]=line[0],line[1],line[2],label
#         train_file.append(instance)
        new_lines=fix_glue_error(line,4)
        for i,new_line in enumerate(new_lines):
#             print("line fixed:",new_line)
            lb=new_line[3]
            if lb not in labels:
                labels.append(lb)
            label = labels.index(lb)   
            instance = dict()
            instance["index"],instance['seq1'],instance['seq2'],instance["label"]=new_line[0],new_line[1],new_line[2],label
            train_file.append(instance)
    print(len(train_file))

skip first line
104743


In [6]:
with open("./dev.tsv","r",encoding="utf-8") as dev:
    t=csv.reader(dev,delimiter='\t')
    for i,line in enumerate(t):
        if i==0:
            print("skip first line")
            continue
#         try:
#             assert len(line) == 4
#         except:
# #             print("error_line:",line)
#             new_lines=fix_glue_error(line,4)
#             for i,new_line in enumerate(new_lines):
#                 print("line fixed:",new_line)
#                 lb=new_line[3]
#                 if lb not in labels:
#                     labels.append(lb)
#                 label = labels.index(lb)   
#                 instance = dict()
#                 instance["index"],instance['seq1'],instance['seq2'],instance["label"]=new_line[0],new_line[1],new_line[2],label
#                 dev_file.append(instance)
#             continue
#         lb=line[3]
#         if lb not in labels:
#             labels.append(lb)
#         label = labels.index(lb)
#         instance = dict()
#         instance["index"],instance['seq1'],instance['seq2'],instance["label"]=line[0],line[1],line[2],label
#         dev_file.append(instance)
        new_lines=fix_glue_error(line,4)
        for i,new_line in enumerate(new_lines):
#             print("line fixed:",new_line)
            lb=new_line[3]
            if lb not in labels:
                labels.append(lb)
            label = labels.index(lb)   
            instance = dict()
            instance["index"],instance['seq1'],instance['seq2'],instance["label"]=new_line[0],new_line[1],new_line[2],label
            dev_file.append(instance)
    print(len(dev_file))

skip first line
5463


### idx from 862 to 870 has format error

In [7]:
with open("./test.tsv","r",encoding="utf-8",errors="ignore") as test:
    t=csv.reader(test,delimiter='\t')
    for i,line in enumerate(t):
        if i==0:
            print("skip first line")
            continue
#         try:
#             assert len(line) == 3
#         except:
# #             print("error_line:",line)
#             new_lines=fix_glue_error(line,3)
#             for i,new_line in enumerate(new_lines):
#                 print("line fixed:",new_line)
#                 instance = dict()
#                 instance["index"],instance['seq1'],instance['seq2'],instance["label"]=new_line[0],new_line[1],new_line[2],None
#                 test_file.append(instance)
#             continue
#         instance = dict()
#         instance["index"],instance['seq1'],instance['seq2'],instance["label"]=line[0],line[1],line[2],None
#         test_file.append(instance)
        new_lines=fix_glue_error(line,3)
        for i,new_line in enumerate(new_lines):
#             print("line fixed:",new_line)
            instance = dict()
            instance["index"],instance['seq1'],instance['seq2'],instance["label"]=new_line[0],new_line[1],new_line[2],None
            try:
                t=int(new_line[0])
            except:
                print("error:",new_line)
            test_file.append(instance)
    print(len(test_file))

skip first line
5463


In [8]:
labels

['not_entailment', 'entailment']

In [9]:
def simple_filt(seq:str):
    if seq is None:
        return None
    tokens = seq.split()
    tokens = [t for t in tokens if t != ""]
    
    return " ".join(tokens)

In [10]:
def write_to_json(list_files:list,file_names:list,labels:list):
    ''' write cached object to json file(train/dev/test)
    and also generate data statistic information
    
    assert the order of input list is train/dev/test
    '''
    def seq_len(seq:str):
        try:
            return len(seq.split(" "))
        except:
            if seq is None:
                return 0
            else:
                raise KeyError
                
    max_lens = []
    for i,list_file in enumerate(list_files):
        max_len = 0
        new_list = []
        for item in list_file:
            item['seq1'],item['seq2'] = simple_filt(item["seq1"]),simple_filt(item['seq2'])
            max_len_two_seq=max(seq_len(item['seq1']),seq_len(item['seq2']))
            max_len = max(max_len_two_seq,max_len)
            new_list.append(item)
        with open(file_names[i],"w") as f:
            json.dump(new_list,f)
        
        print("successfully dump %s file" %file_names[i])
        max_lens.append(max_len)
    
    data_info = dict()
    train_info = dict()
    dev_info = dict()
    test_info = dict()
    idx2lb = dict()
    for i,lb in enumerate(labels):
        idx2lb[i] = lb
    print("index to label:",idx2lb)
    
    train_info["instance_num"],train_info["instance_max_len"] = len(list_files[0]),max_lens[0]
    dev_info["instance_num"],dev_info["instance_max_len"] = len(list_files[1]),max_lens[1]
    test_info["instance_num"],test_info["instance_max_len"] = len(list_files[2]),max_lens[2]
    
    
    data_info["labels"] = idx2lb
    data_info["train"] = train_info
    data_info["dev"] = dev_info
    data_info["test"] = test_info
    
    data_info_name = "./data_info.json"
    with open(data_info_name,"w") as f:
        json.dump(data_info,f)
    print("successfully dump %s file" %data_info_name )

In [11]:
labels

['not_entailment', 'entailment']

In [12]:
write_to_json([train_file,dev_file,test_file],["train.json","dev.json","test.json"],labels)

successfully dump train.json file
successfully dump dev.json file
successfully dump test.json file
index to label: {0: 'not_entailment', 1: 'entailment'}
successfully dump ./data_info.json file
